<a href="https://colab.research.google.com/github/1PgansU/Research_-/blob/main/%E6%B7%B1%E5%BA%A6%E5%AD%A6%E4%B9%A0_%E6%9C%89%E6%95%88%E6%8A%97%E7%94%9F%E7%B4%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Define custom wrapper metrics

### Subtask:
Define `ChempropBinaryAccuracy` and `ChempropBinaryAUROC` classes that inherit from `torchmetrics.classification.BinaryAccuracy` and `torchmetrics.classification.BinaryAUROC` respectively. These wrappers will override the `update` method to accept all arguments passed by the `chemprop` model's `_evaluate_batch` method, but only forward the `preds` and `target` to the parent `torchmetrics` update method.


**Reasoning**:
The previous code failed because the `torchmetrics` update method received too many arguments. To fix this, I need to create custom wrapper classes for `BinaryAccuracy` and `BinaryAUROC` that override the `update` method to accept all arguments from `chemprop` but only pass `preds` and `target` to the parent `torchmetrics` update method.



In [ ]:
import torch
import torchmetrics

class ChempropBinaryAccuracy(torchmetrics.classification.BinaryAccuracy):
    """Wrapper for BinaryAccuracy to accept chemprop's _evaluate_batch arguments."""
    def update(self, preds: torch.Tensor, target: torch.Tensor, *args, **kwargs) -> None:
        # chemprop's _evaluate_batch passes preds, targets, mask, weights, lt_mask, gt_mask
        # BinaryAccuracy.update expects only preds and target
        super().update(preds, target)

class ChempropBinaryAUROC(torchmetrics.classification.BinaryAUROC):
    """Wrapper for BinaryAUROC to accept chemprop's _evaluate_batch arguments."""
    def update(self, preds: torch.Tensor, target: torch.Tensor, *args, **kwargs) -> None:
        # chemprop's _evaluate_batch passes preds, targets, mask, weights, lt_mask, gt_mask
        # BinaryAUROC.update expects only preds and target
        super().update(preds, target)

print("Defined custom ChempropBinaryAccuracy and ChempropBinaryAUROC classes.")

Defined custom ChempropBinaryAccuracy and ChempropBinaryAUROC classes.


**Reasoning**:
The custom wrapper classes for the metrics have been defined. Now, I need to replace the original `torchmetrics.classification.BinaryAccuracy` and `torchmetrics.classification.BinaryAUROC` instances with these new custom classes in the `metric_list` of the notebook. Then, the training and testing process needs to be re-initiated.



In [ ]:
import os

# 在 Colab 中自动安装 chemprop
if os.getenv("COLAB_RELEASE_TAG"):
    try:
        import chemprop
    except ImportError:
        !git clone https://github.com/chemprop/chemprop.git
        %cd chemprop
        !pip install .
        %cd ..

# ========== 1. 导入依赖 ==========
from pathlib import Path
from lightning import pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint
import pandas as pd
import rdkit.Chem as Chem # Added import for RDKit
from chemprop import data, featurizers, models, nn
import torchmetrics
import torch

# Define custom wrapper metrics as per the subtask
class ChempropBinaryAccuracy(torchmetrics.classification.BinaryAccuracy):
    """Wrapper for BinaryAccuracy to accept chemprop's _evaluate_batch arguments."""
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.alias = "accuracy" # Added alias attribute

    def update(self, preds: torch.Tensor, target: torch.Tensor, *args, **kwargs) -> None:
        # chemprop's _evaluate_batch passes preds, targets, mask, weights, lt_mask, gt_mask
        # BinaryAccuracy.update expects only preds and target
        super().update(preds, target)

class ChempropBinaryAUROC(torchmetrics.classification.BinaryAUROC):
    """Wrapper for BinaryAUROC to accept chemprop's _evaluate_batch arguments."""
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.alias = "auroc" # Added alias attribute

    def update(self, preds: torch.Tensor, target: torch.Tensor, *args, **kwargs) -> None:
        # chemprop's _evaluate_batch passes preds, targets, mask, weights, lt_mask, gt_mask
        # BinaryAUROC.update expects only preds and target
        super().update(preds, target)

# ========== 2. 配置输入数据（抗生素活性） ==========
# 假设你的 CSV 文件包含两列：'smiles' 和 'activity'（1=有活性，0=无活性）
from google.colab import drive
drive.mount('/content/drive')
input_path = "/content/drive/MyDrive/chemprop/train.csv"
num_workers = 0 # number of workers for dataloader. 0 means using main process for data loading
smiles_column = 'smiles' # name of the column containing SMILES strings
target_columns = ['target'] # list of names of the columns containing targets

# ========== 3. 加载数据 ==========
df_input = pd.read_csv(input_path)

# --- 修复：过滤无效的SMILES字符串 ---
def is_valid_smiles(smi):
    """Checks if a SMILES string can be parsed by RDKit."""
    try:
        mol = Chem.MolFromSmiles(smi, sanitize=True)
        return mol is not None
    except Exception:
        return False

initial_data_size = len(df_input)
valid_smiles_mask = df_input[smiles_column].apply(is_valid_smiles)
df_input_filtered = df_input[valid_smiles_mask].copy() # Use .copy() to avoid SettingWithCopyWarning

num_invalid_smiles = initial_data_size - len(df_input_filtered)
if num_invalid_smiles > 0:
    print(f"警告：检测到并移除了 {num_invalid_smiles} 个无效的 SMILES 字符串。")
    # Optionally, print the invalid SMILES for debugging:
    # invalid_smiles_examples = df_input[~valid_smiles_mask][smiles_column].tolist()[:5]
    # print("无效的 SMILES 字符串示例:", invalid_smiles_examples)
# --- 修复结束 ---

smis = df_input_filtered[smiles_column].values
ys = df_input_filtered[target_columns].values  # 应为 0 或 1

# ========== 4. 构建 MoleculeDatapoint ==========
all_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(smis, ys)]

# ========== 5. 数据划分（80/10/10） ==========
mols = [d.mol for d in all_data]
train_indices, val_indices, test_indices = data.make_split_indices(
    mols, "random", (0.8, 0.1, 0.1)
)
train_data, val_data, test_data = data.split_data_by_indices(
    all_data, train_indices, val_indices, test_indices
)

# ========== 6. 构建数据集（分类任务无需归一化目标！） ==========
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

# 分类任务：target 不归一化！
train_dset = data.MoleculeDataset(train_data[0], featurizer)
val_dset = data.MoleculeDataset(val_data[0], featurizer)
test_dset = data.MoleculeDataset(test_data[0], featurizer)

# ========== 7. 创建 DataLoader ==========
train_loader = data.build_dataloader(train_dset, num_workers=num_workers)
val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)
test_loader = data.build_dataloader(test_dset, num_workers=num_workers, shuffle=False)

# ========== 8. 构建二分类 D-MPNN 模型 ==========
# (1) 消息传递：以键为单位 → 实现 D-MPNN 的有向消息传递
mp = nn.BondMessagePassing()

# (2) 聚合：使用 SUM（与原始论文一致）
agg = nn.SumAggregation()  # ← 关键修改：论文用 sum pooling

# (3) 预测头：二分类，输出概率
ffn = nn.BinaryClassificationFFN()  # 自动使用 BCE 损失 + sigmoid 输出

# (4) 其他配置
batch_norm = True
# 分类任务常用指标：准确率、AUC
metric_list = [ChempropBinaryAccuracy(), ChempropBinaryAUROC()]  # Using custom wrapper metrics

# 组装模型
mpnn = models.MPNN(mp, agg, ffn, batch_norm, metric_list)

# ========== 9. 配置训练器 ==========
checkpointing = ModelCheckpoint(
    "checkpoints",
    "best-{epoch}-{val_loss:.3f}",
    monitor="val_loss",
    mode="min",
    save_last=True,
)

trainer = pl.Trainer(
    logger=False,
    enable_checkpointing=True,
    accelerator="auto",
    devices=1,
    max_epochs=30,  # 通常分类任务收敛更快
    callbacks=[checkpointing],
)

# ========== 10. 训练与测试 ==========
trainer.fit(mpnn, train_loader, val_loader)

# Add custom classes to safe globals for unpickling, as suggested by the error message.
import sys
# Dynamically add the classes from the current module (__main__) to safe globals.
# This is a workaround for the UnpicklingError with custom classes.
# Note: This approach allows unpickling, but it's important to ensure the source is trusted.
if hasattr(sys.modules['__main__'], 'ChempropBinaryAccuracy') and \
   hasattr(sys.modules['__main__'], 'ChempropBinaryAUROC'):
    torch.serialization.add_safe_globals([
        sys.modules['__main__'].ChempropBinaryAccuracy,
        sys.modules['__main__'].ChempropBinaryAUROC
    ])
else:
    print("Warning: Custom metric classes not found in __main__ for safe global registration.")

results = trainer.test(dataloaders=test_loader, ckpt_path='best', weights_only=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[04:41:15] Explicit valence for atom # 16 Al, 6, is greater than permitted


警告：检测到并移除了 1 个无效的 SMILES 字符串。


INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.12/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:881: Checkpoint directory /content/checkpoints exists and is not empty.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loading `train_dataloader` to estimate number of stepping batches.
INFO:lightning.pytorch.utilities.rank_zero:Loading `train_dataloader` to estimate number of stepping batches.


┏━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name            ┃ Type                    ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ message_passing │ BondMessagePassing      │  227 K │ train │     0 │
│ 1 │ agg             │ SumAggregation          │      0 │ train │     0 │
│ 2 │ bn              │ BatchNorm1d             │    600 │ train │     0 │
│ 3 │ predictor       │ BinaryClassificationFFN │ 90.6 K │ train │     0 │
│ 4 │ X_d_transform   │ Identity                │      0 │ train │     0 │
│ 5 │ metrics         │ ModuleList              │      0 │ train │     0 │
└───┴─────────────────┴─────────────────────────┴────────┴───────┴───────┘

...

Output()

INFO: `Trainer.fit` stopped: `max_epochs=30` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.


INFO: Restoring states from the checkpoint path at /content/checkpoints/best-epoch=12-val_loss=0.125.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at /content/checkpoints/best-epoch=12-val_loss=0.125.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at /content/checkpoints/best-epoch=12-val_loss=0.125.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at /content/checkpoints/best-epoch=12-val_loss=0.125.ckpt


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/accuracy       │    0.9658119678497314     │
│        test/auroc         │    0.8694690465927124     │
└───────────────────────────┴───────────────────────────┘

In [ ]:
# ========== 保存最终模型 ==========
# 保存为 .ckpt 文件（包含模型结构 + 权重 + 训练状态）
final_model_path = "/content/drive/MyDrive/chemprop/final_model.ckpt"
trainer.save_checkpoint(final_model_path)
print(f"✅ 模型已保存至: {final_model_path}")

INFO: `weights_only` was not set, defaulting to `False`.
INFO:lightning.pytorch.trainer.connectors.checkpoint_connector:`weights_only` was not set, defaulting to `False`.


✅ 模型已保存至: /content/drive/MyDrive/chemprop/final_model.ckpt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls -l "/content/drive/MyDrive/chemprop/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 4287
-rw------- 1 root root 3851390 Jan  6 04:42 final_model.ckpt
-rw------- 1 root root  427550 Jan  4 12:38 predict.csv
-rw------- 1 root root  109315 Jan  4 12:30 train.csv


In [ ]:
import os

# 在 Colab 中自动安装 chemprop
if os.getenv("COLAB_RELEASE_TAG"):
    try:
        import chemprop
    except ImportError:
        !git clone https://github.com/chemprop/chemprop.git
        %cd chemprop
        !pip install .
        %cd ..

# ========== 加载最佳模型进行测试与绘图 ==========
# 使用验证集上表现最好的模型（由 ModelCheckpoint 自动保存）
from google.colab import drive
drive.mount('/content/drive')

# Define paths for both explicitly saved model and ModelCheckpoint's best model
final_model_drive_path = "/content/drive/MyDrive/chemprop/final_model.ckpt"
checkpoint_dir = "/content/checkpoints"

# -*- coding: utf-8 -*-
"""
预测候选分子活性并生成排名 Excel 表格
"""

import os
from google.colab import drive
drive.mount('/content/drive')

# ========== 1. 导入依赖 ==========
import pandas as pd
import torch
import rdkit.Chem as Chem
from chemprop import data, featurizers, models
import numpy as np
import torchmetrics # Added import for torchmetrics

# ========== 2. 配置路径 ==========
model_path = "/content/drive/MyDrive/chemprop/final_model.ckpt"       # 你的模型
input_csv = "/content/drive/MyDrive/chemprop/predict.csv"          # 候选分子
output_excel = "/content/drive/MyDrive/chemprop/ranked_predict.xlsx"  # 输出 Excel

# ========== 3. 加载模型 ==========
print("🔧 正在加载模型...")

# Add custom classes to safe globals for unpickling, as suggested by the error message.
import sys
# Dynamically add the classes from the current module (__main__) to safe globals.
# This is a workaround for the UnpicklingError with custom classes.
# Note: This approach allows unpickling, but it's important to ensure the source is trusted.
# For this to work, ChempropBinaryAccuracy and ChempropBinaryAUROC classes need to be defined here or imported.
# Since they are not, we will re-define them here for the prediction script to function.

# Re-define custom wrapper metrics to ensure they are available in this script's scope
# This is necessary because the previous definitions were in a different execution cell
class ChempropBinaryAccuracy(torchmetrics.classification.BinaryAccuracy):
    """Wrapper for BinaryAccuracy to accept chemprop's _evaluate_batch arguments."""
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.alias = "accuracy"

    def update(self, preds: torch.Tensor, target: torch.Tensor, *args, **kwargs) -> None:
        super().update(preds, target)

class ChempropBinaryAUROC(torchmetrics.classification.BinaryAUROC):
    """Wrapper for BinaryAUROC to accept chemprop's _evaluate_batch arguments."""
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.alias = "auroc"

    def update(self, preds: torch.Tensor, target: torch.Tensor, *args, **kwargs) -> None:
        super().update(preds, target)

# Now add them to safe globals
if hasattr(sys.modules['__main__'], 'ChempropBinaryAccuracy') and \
   hasattr(sys.modules['__main__'], 'ChempropBinaryAUROC'):
    torch.serialization.add_safe_globals([
        sys.modules['__main__'].ChempropBinaryAccuracy,
        sys.modules['__main__'].ChempropBinaryAUROC
    ])
else:
    print("Warning: Custom metric classes not found in __main__ for safe global registration.")

model = models.MPNN.load_from_checkpoint(model_path, strict=False)
model.eval()
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()  # 必须与训练时一致

# ========== 4. 加载候选分子 ==========
print("📥 正在读取候选分子...")
df_candidates = pd.read_csv(input_csv)

# 清洗：只保留合法 SMILES
def is_valid_smiles(smi):
    try:
        return Chem.MolFromSmiles(smi) is not None
    except:
        return False

valid_mask = df_candidates['SMILES'].apply(is_valid_smiles)
df_valid = df_candidates[valid_mask].copy()
print(f"✅ 共 {len(df_valid)} 个有效分子（{len(df_candidates) - len(df_valid)} 个无效 SMILES 已过滤）")

# ========== 5. 批量预测 ==========
print("🧠 正在预测活性概率...")
predictions = []

# 构建预测数据集（无标签）
# To resolve AttributeError: 'TrainingBatch' object has no attribute 'V'
# when y is None, it appears to create an incomplete TrainingBatch or BatchMolGraph.
# Providing a dummy target forces a complete TrainingBatch creation.
pred_datapoints = [data.MoleculeDatapoint.from_smi(smi, y=0) for smi in df_valid['SMILES']]

# The featurizer should be passed to MoleculeDataset, which will handle featurization.
# Remove the redundant manual featurization loop.
pred_dset = data.MoleculeDataset(pred_datapoints, featurizer) # Corrected: Pass featurizer here

# 重要：创建 DataLoader（注意 batch_size 可调，但太大可能 OOM）
pred_loader = data.build_dataloader(
    pred_dset,
    batch_size=32,  # 可根据内存调整
    num_workers=0,
    shuffle=False
)

with torch.no_grad():
    for batch in pred_loader:
        logits = model(batch.bmg) # Corrected: Pass batch.bmg to the model
        probs = torch.sigmoid(logits).cpu().numpy().flatten()
        predictions.extend(probs)

# ========== 6. 生成排名表格 ==========
df_valid['prediction_score'] = predictions
df_valid['rank'] = df_valid['prediction_score'].rank(method='min', ascending=False).astype(int)

# 按预测分降序排列
df_ranked = df_valid.sort_values('prediction_score', ascending=False).reset_index(drop=True)

# ========== 7. 保存为 Excel ==========
df_ranked.to_excel(output_excel, index=False)
print(f"🎯 排名结果已保存至: {output_excel}")
print("\n📋 前 500 名候选分子:")
print(df_ranked[['rank', 'prediction_score', 'SMILES']].head(500)) # Corrected 'smiles' to 'SMILES')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔧 正在加载模型...


/usr/local/lib/python3.12/dist-packages/lightning/pytorch/core/saving.py:197: Found keys that are not in the model state dict but in the checkpoint: ['metrics.0.task_weights', 'metrics.1.task_weights']


📥 正在读取候选分子...
✅ 共 6820 个有效分子（0 个无效 SMILES 已过滤）
🧠 正在预测活性概率...
🎯 排名结果已保存至: /content/drive/MyDrive/chemprop/ranked_predict.xlsx

📋 前 500 名候选分子:
     rank  prediction_score                                             SMILES
0       1          0.730688  Nc1nc(-n2cc(C(=O)O)c(=O)c3cc(F)c(NC[C@@H](O)CO...
1       2          0.730673  Nc1nc(-n2cc(C(=O)O)c(=O)c3cc(F)c(NC[C@H](O)COC...
2       3          0.729484  C[C@@H]1COc2c(N3CCN(C)CC3)c(F)cc3c(=O)c(C(=O)n...
3       4          0.729439  C[C@H]1COc2c(N3CCN(C)CC3)c(F)cc3c(=O)c(C(=O)n4...
4       5          0.729387  [N-]=[N+]=N[C@H]1[C@@H](O)[C@@H](n2cc(F)c(=O)[...
..    ...               ...                                                ...
495   496          0.713583  Cc1cc(NC(=O)c2cn3c4c(cc(F)c(N5CCN(C)CC5)c4OC[C...
496   497          0.713563  CCN(C(=O)c1cn2c3c(cc(F)c(N4CCN(C)CC4)c3OC[C@@H...
497   498          0.713539  CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)N3CCC[C@@](...
498   499          0.713474  COC(=O)[C@H](C)NC(=S)N1CCN(c2c(F)cc3c(=O

## 总结一下你做的修改，并提醒编码时需要注意的问题

在完成你的任务过程中，我们共同解决了多个问题。以下是本次任务中主要的修改总结和一些编码注意事项：

### 解决的问题及修改方案：

1.  **SMILES 字符串有效性检查 (`RuntimeError: SMILES ... is invalid!`):**
    *   **问题**: 原始代码没有过滤掉无效的 SMILES 字符串，导致 RDKit 无法解析，模型训练失败。
    *   **修改**: 增加了 `is_valid_smiles` 函数，在加载数据后，使用 RDKit 检查 SMILES 字符串的有效性，并移除了无效的条目。`df_input_filtered` 现在只包含有效的分子数据。

2.  **指标模块引入错误 (`AttributeError: module 'chemprop.nn.metrics' has no attribute 'Accuracy'/'ROC_AUC'`):**
    *   **问题**: `chemprop.nn.metrics` 中没有直接暴露 `Accuracy` 和 `ROC_AUC` 属性，导致无法正确导入。
    *   **修改**: 将指标替换为 `torchmetrics.classification.BinaryAccuracy` 和 `torchmetrics.classification.BinaryAUROC`，这是 PyTorch 生态系统中更常用和标准的二分类指标实现。

3.  **自定义指标 `alias` 属性缺失 (`AttributeError: 'ChempropBinaryAccuracy' object has no attribute 'alias'`):**
    *   **问题**: `chemprop` 的日志机制在 `_evaluate_batch` 方法中期望自定义指标有一个 `alias` 属性来命名，但自定义的 `ChempropBinaryAccuracy` 和 `ChempropBinaryAUROC` 缺少此属性。
    *   **修改**: 在 `ChempropBinaryAccuracy` 和 `ChempropBinaryAUROC` 类的 `__init__` 方法中添加了 `self.alias = "accuracy"` 和 `self.alias = "auroc"`。

4.  **自定义类反序列化错误 (`UnpicklingError` / `AttributeError: Can't get attribute 'ChempropBinaryAccuracy'`):**
    *   **问题**: 在加载保存的模型（checkpoint）时，由于 PyTorch 2.0+ 版本的 `torch.load` 默认 `weights_only=True`，并且自定义类未注册，导致无法反序列化模型中包含的自定义指标类。
    *   **修改**: 在 `trainer.test` 调用中明确设置 `weights_only=False`。更重要的是，在模型加载之前，使用 `torch.serialization.add_safe_globals()` 将自定义的 `ChempropBinaryAccuracy` 和 `ChempropBinaryAUROC` 类添加到 PyTorch 的安全全局列表中，允许它们被正确地反序列化。

5.  **模型 checkpoint 路径查找错误 (`FileNotFoundError` / `IndexError: list index out of range`):**
    *   **问题**: 在测试和预测阶段，有时 `glob.glob` 无法找到 `best-*.ckpt` 文件，或者路径设置有误。
    *   **修改**: 调整了模型加载逻辑，首先尝试从 `trainer.checkpoint_callback.best_model_path` 直接获取最佳模型路径。如果获取失败，则回退到 `glob.glob` 搜索 `/content/checkpoints/` 目录。此外，还添加了对明确保存到 Google Drive 的 `final_model.ckpt` 的检查，提供了更鲁棒的模型加载策略。

6.  **`os`、`models`、`torchmetrics` 模块未定义 (`NameError: name 'os' is not defined` 等):**
    *   **问题**: 在一些独立的代码单元格中，所需的模块（如 `os`、`chemprop.models`、`torchmetrics`）没有被显式导入，导致 `NameError` 或 `ModuleNotFoundError`。
    *   **修改**: 在使用这些模块或其组件之前，确保在每个相关代码单元格的开头添加了相应的 `import` 语句。

7.  **预测数据 `TrainingBatch` 属性缺失 (`AttributeError: 'TrainingBatch' object has no attribute 'V'`):**
    *   **问题**: 在预测阶段，当 `MoleculeDatapoint.from_smi` 在没有 `y` 值的情况下被调用时，有时会导致 `TrainingBatch` 对象不完整，缺失 `V` (顶点特征) 等属性。
    *   **修改**: 在创建 `pred_datapoints` 时，显式为 `MoleculeDatapoint.from_smi` 传入一个 `y=0` 的哑标签。这确保了 `TrainingBatch` 对象能被完整地构建。同时，在调用 `model` 进行预测时，从 `batch` 中提取 `batch.bmg` (BatchMolGraph) 传递给模型，因为 `model` 的 `forward` 方法期望的是 `BatchMolGraph` 对象。

8.  **`MoleculeDataset` featurizer 传入错误 (`TypeError: 'NoneType' object is not callable`):**
    *   **问题**: 在预测阶段，`MoleculeDataset` 被错误地传入 `featurizer=None`，而 `MoleculeDataset` 需要一个 featurizer 来处理分子数据。
    *   **修改**: 在创建 `pred_dset` 时，将 `featurizer` 对象直接传入 `data.MoleculeDataset(pred_datapoints, featurizer)`，移除了之前尝试手动预 featurize 的冗余循环，让 `MoleculeDataset` 自己完成 featurization。

9.  **结果 DataFrame 列名错误 (`KeyError: "['smiles'] not in index"`):**
    *   **问题**: 在打印最终结果时，尝试访问名为 `'smiles'` (小写) 的列，但 DataFrame 中实际的列名是 `'SMILES'` (大写)。
    *   **修改**: 将打印语句中的列名修正为 `'SMILES'`，以匹配 DataFrame 中的实际列名。

### 编码时需要注意的问题：

*   **显式导入**: 任何在代码单元格中使用的库、模块或自定义类，都应该在单元格的开头显式导入。不要假定上一个单元格的导入在当前单元格中仍然有效，尤其是在 Colab 这种可能存在会话重置的环境中。
*   **大小写敏感**: Python 对变量名、函数名、类名、字典键和 DataFrame 列名的大小写是敏感的。始终确保使用正确的名称。
*   **数据结构理解**: 深入理解你正在处理的数据结构（例如，`MoleculeDatapoint`、`TrainingBatch`、`BatchMolGraph` 等）以及它们包含的属性。当遇到 `AttributeError` 时，这意味着你尝试访问的属性不存在于该对象上。通过查阅文档或使用 `dir(obj)` 可以帮助你了解对象可用的属性。
*   **PyTorch/Lightning 序列化**: 当使用 PyTorch Lightning 保存和加载包含自定义类的模型时，需要特别注意序列化问题：
    *   确保自定义类（如自定义指标）的定义在加载模型时是可用的。
    *   对于 PyTorch 2.0+ 版本，如果自定义类无法直接通过 `torch.load` 加载，可能需要使用 `torch.serialization.add_safe_globals()` 注册这些类，并可能需要在 `load_from_checkpoint` 中设置 `strict=False` 以忽略非模型参数的键不匹配警告。
*   **DataLoader 和迭代器**: `DataLoader` 对象是迭代器。一旦遍历完成，它就耗尽了。如果需要多次使用相同的数据加载器，你可能需要重新创建它或者确保它能够重置。
*   **错误信息分析**: 仔细阅读错误信息，尤其是 Traceback。它们通常会精确指出问题发生的位置和原因，例如 `AttributeError`、`KeyError`、`NameError` 或 `TypeError`。
*   **环境一致性**: 确保开发和部署环境中的库版本一致，以避免因版本差异引起的问题。对于 Colab，在需要时重复安装依赖可以保证这一点。

希望这些总结能帮助你在未来的项目中避免类似的问题！